<a href="https://colab.research.google.com/github/skramer-dev/ai-lab/blob/main/02_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Klassifikation - Sentiment Analyse

In diesem Notebook möchten wir uns mit der Klassifikation von Texten beschäftigen. Vereinfacht gesagt beschäftigt sich Sentiment Analysis damit, natürlichsprachliche Aussagen dahingehend zu bewerten, ob die subjektive Aussage des Sprechers positiv oder negativ wertend gemeint ist.

Zu diesem Zweck haben wir den Datensatz von _Sentiment140_, einem Projekt der Stanford University, ausgewählt. Er beinhaltet 16 Millionen Tweets, die aufgrund der enthaltenen Emoticons automatisch in positiv und negativ eingeteilt wurden.

---

Dieses Notebook gliedert sich in die folgenden Teile:

1. Datenanalyse und Preprocessing
2. Tokenisierung & Vokabular
3. Klassifikation
4. Klassifikation mit huggingface transformers

In [2]:
import tensorflow as tf
tf.__version__

'2.9.2'

## 0. Vorbereitung

Als erstes muss der Datensatz von http://help.sentiment140.com/for-students in diese Colab Virtual Machine geladen werden.

In [5]:
! gdown 0B04GJPshIjmPRnZManQwWEdTZjg
DATA_DIR = '/content/sentiment140'
!unzip /content/trainingandtestdata.zip -d {DATA_DIR}
%rm /content/trainingandtestdata.zip
%ls -la /{DATA_DIR}

Archive:  /content/trainingandtestdata.zip
  inflating: /content/sentiment140/testdata.manual.2009.06.14.csv  
  inflating: /content/sentiment140/training.1600000.processed.noemoticon.csv  
total 233292
drwxr-xr-x 2 root root      4096 Nov 16 11:08 ./
drwxr-xr-x 1 root root      4096 Nov 16 11:08 ../
-rw-r--r-- 1 root root     74326 Mar  4  2010 testdata.manual.2009.06.14.csv
-rw-r--r-- 1 root root 238803811 Mar  4  2010 training.1600000.processed.noemoticon.csv


## 1. Datenbeschaffung und -Analyse

Im ersten Schritt sollen folgende Schritte durchgeführt werden.

In [6]:
import pandas as pd
import numpy as np
import os

1. Ladet den Datensatz in einen Pandas Dataframe. Welche Feature gibt es? Wie viele Samples gibt es?

2. Da wir uns nur für die Felder `polarity` und `text` interessieren, sollte die Liste mit den Daten folgendes Format haben : `id => (polarity, text)`. 

3. Wandelt diese Werte in 1 (positiv) und 0 (negativ).

4. Fügt eine Spalte für die Anzahl an Wörtern im Text hinzu.

5. Analysiert den Datensatz mit den Pandas Boardmitteln.

In [7]:
column_names = ["polarity","id","date","query","user","text"]
df = pd.read_csv('/content/sentiment140/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names=column_names)
df.drop(['date', 'query', 'user'], axis=1, inplace=True)
df["polarity"] = df["polarity"].apply(lambda x: 0 if x==0 else 1)

In [9]:
df.iloc[-1]

polarity                                                    1
id                                                 2193602129
text        happy #charitytuesday @theNSPCC @SparksCharity...
Name: 1599999, dtype: object

In [10]:
df['word_count'] = df['text'].str.count(' ') + 1
print(f"Mean tweet length: {df.word_count.mean()}")

Mean tweet length: 14.382130625


## 2. Tokenisierung

Um einen Einblick in die Daten zu bekommen und um später ein Modell zur Sentiment Analyse trainieren zu können, sollen die Daten nun aufbereitet werden. Da die sinntragenden Elemente in den Tweets die Wörter sind, sollten Sie die Tweets in Wörter aufteilen. Um genau zu sein, ist der Term 'Wörter' hier aus linguistischer Sicht etwas falsch, man spricht eigentlich von Tokens. Daher nennt man das Aufteilen von Text auch Tokenizing und die Funktion, die sowas kann, Tokenizer.

1. Der allereinfachste Tokenizer ist vermutlich die `split` Methode. Tokenisiert damit die eingelesen Tweets. Am Ende solltet ihr eine Liste `tokenized = [(polarity, [token_1,token_2, ...])]` erhalten.

In [11]:
df['tokenized'] = df['text'].str.split(' ')

In [12]:
pd.set_option("display.max_colwidth", None)
print(df.iloc[-1].to_string())

polarity                                                                          1
id                                                                       2193602129
text                 happy #charitytuesday @theNSPCC @SparksCharity @SpeakingUpH4H 
word_count                                                                        6
tokenized     [happy, #charitytuesday, @theNSPCC, @SparksCharity, @SpeakingUpH4H, ]


Abgesehen von natürlichsprachlichen Wörtern sind in Tweets mindestens auch Hashtags, Mentions und Links enthalten. Überlegt euch, ob es Sinn ergibt, alle diese Bestandteile in den Daten in dieser From zu behalten. Begründet kurz Ihre Entscheidungen.
Falls ihr euch entschlossen habt, nicht alle diese Bestandteile zu behalten, filtert dementsprechend eure Daten. Die Struktur Ihrer Daten sollte am Ende gleich bleiben: `df['cleaned'] = [token_1,...]`

Mentions und gewisse Hashtags können durchaus Einfluss auf das Sentiment der Aussage haben

Zählt die Tokens in eurem Datensatz. Benutzt dafür ein Dictionary oder andere Python Build-Ins. 

1. Wieviele unterschiedliche Wörter gibt es?
2. Gebt die 100 häufigsten Wörter sortiert aus. 

Was zieht ihr aus den beiden Analysen? Was müsst ihr zusätzlich noch filtern?

In [13]:
tokens = df['tokenized']
print(tokens[:3])
tokens = [element for sublist in tokens for element in sublist]
print(f"number of tokens: {len(tokens)}")
unique_words = list(set(tokens))
num_unique_words = len(unique_words)
print(f"number of unique words: {num_unique_words}")

0    [@switchfoot, http://twitpic.com/2y1zl, -, Awww,, that's, a, bummer., , You, shoulda, got, David, Carr, of, Third, Day, to, do, it., ;D]
1      [is, upset, that, he, can't, update, his, Facebook, by, texting, it..., and, might, cry, as, a, result, , School, today, also., Blah!]
2                               [@Kenichan, I, dived, many, times, for, the, ball., Managed, to, save, 50%, , The, rest, go, out, of, bounds]
Name: tokenized, dtype: object
number of tokens: 23011409
number of unique words: 1350544


In [14]:
from collections import Counter
Counter(" ".join(df["text"]).split()).most_common(100)

[('to', 552962),
 ('I', 496619),
 ('the', 487501),
 ('a', 366212),
 ('my', 280025),
 ('and', 275263),
 ('i', 250016),
 ('is', 217693),
 ('you', 213871),
 ('for', 209801),
 ('in', 202294),
 ('of', 179554),
 ('it', 171812),
 ('on', 154365),
 ('have', 132249),
 ('so', 125155),
 ('me', 122509),
 ('that', 118685),
 ('with', 110843),
 ('be', 108069),
 ('but', 106272),
 ('at', 102196),
 ("I'm", 99559),
 ('was', 99140),
 ('just', 96284),
 ('not', 88110),
 ('this', 77810),
 ('get', 76734),
 ('like', 73302),
 ('are', 72568),
 ('up', 70007),
 ('all', 67901),
 ('-', 67079),
 ('out', 67030),
 ('go', 62969),
 ('your', 60854),
 ('good', 59775),
 ('day', 55748),
 ('do', 54628),
 ('from', 54182),
 ('got', 53871),
 ('now', 53591),
 ('going', 53236),
 ('love', 50051),
 ('no', 49622),
 ('about', 46708),
 ('work', 45913),
 ('will', 45898),
 ('back', 44033),
 ('u', 43568),
 ("it's", 43422),
 ('some', 42745),
 ('am', 42724),
 ('can', 42506),
 ("don't", 42472),
 ('really', 42152),
 ('had', 41548),
 ('see', 41

Eine Möglichkeit für komplexere Preprocessing-Methoden ist das Entfernen von Stoppwörtern. Hiefür nutzen wir NLTK.

Filtert eure gesäuberten Tokens auf Stopwörter.

In [15]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
df['cleaned'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [16]:
df['tokenized'] = df['cleaned'].str.split(' ')
df['word_count'] = df['cleaned'].str.count(' ') + 1
print(f"Mean tweet length: {df.word_count.mean()}")

Mean tweet length: 8.776623125


In [17]:
print(df.iloc[-3].to_string())

polarity                                                              1
id                                                           2193601991
text          Are you ready for your MoJo Makeover? Ask me for details 
word_count                                                            6
tokenized                   [Are, ready, MoJo, Makeover?, Ask, details]
cleaned                            Are ready MoJo Makeover? Ask details


## 3. Klassifikation

Wie Eingangs erwähnt, beschäftigt sich Sentiment Analysis damit, eine Äußerung automatisch dahingehend zu klassifizieren,
ob der Inhalt positiv oder negativ gemeint ist.
Es handelt sich also um eine binäre Klassifikation.

Für das Training des neuronalen Netzes orintiert sich der nachfolgende Teil an _Keras_ als Framework. Ihr könnt aber auch ein anderes Framework wie bspw. _Pytorch_ benutzen. 

Aktuell liegen unsere Daten zwar in tokenisierter und gesäuberter Form vor, wir müssen unsere Daten aber noch in Vektoren transformieren.

Für die erste Klassifikation encoden wir die Eingabe als **Bag-of-Words**, sodass jedes potentielle Wort einem Eingabeneuron einspricht. _Ein Beispiel_: Zwei Tweets "lorem ipsum" und "foo foo bar", die Vektoren hätten die Länge 4 und für den ersten Tweet wäre der Vektor [1,1,0,0], für den zweiten [0,0,2,1].

### 3.1 Vokabular

Befüllt das dictionary `word2idx` so, dass jedes Wort auf einen Index abgebildet wird und die Indizes streng monoton aufsteigend sind. Für das Beispiel oben wäre `word2idx = {"lorem": 0, "ipsum": 1, "foo": 3, "bar": 4}`

In [18]:
tokens = df['tokenized']
print(tokens[:3])
tokens = [element for sublist in tokens for element in sublist]
print(f"number of tokens: {len(tokens)}")
unique_words = list(set(tokens))
num_unique_words = len(unique_words)
print(f"number of unique words: {num_unique_words}")

0    [@switchfoot, http://twitpic.com/2y1zl, -, Awww,, that's, bummer., You, shoulda, got, David, Carr, Third, Day, it., ;D]
1                          [upset, can't, update, Facebook, texting, it..., might, cry, result, School, today, also., Blah!]
2                                       [@Kenichan, I, dived, many, times, ball., Managed, save, 50%, The, rest, go, bounds]
Name: tokenized, dtype: object
number of tokens: 14042597
number of unique words: 1350424


In [19]:
word2idx = {}
for idx, key in enumerate(unique_words):
    word2idx[key] = idx

In [20]:
word2idx['2']

392990

Welche Länge werden die Vektoren haben?

In [21]:
VECTOR_LEN = 1350424
BATCH_SIZE = 100

Wir könnten mit `numpy` ein Array befüllen, das für jeden der 16 Millionen Tweets einen Vektor wie oben beschrieben enthält. 
Bevor ihr damit beginnen, überschlagt, wieviel Speicherplatz (im Hauptspeicher) ein solches Array belegen würde, wenn jeder Eintrag 32 bit hat. Reicht euer Hauptspeicher dafür aus?

In [22]:
MEMORY = 32 * 1350424 * 16000000
print(f"{MEMORY / 8000000} Mbyte")

86427136.0 Mbyte


### 3.2 Data Generator
Um das Problem mit dem zu kleinen Hauptspeicher zu umgehen, bietet Keras die Möglichkeit, anstatt auf einem kompletten Datensatz zu operieren, immer nur kleinere Häppchen abzuarbeiten. Dazu wird ein Python-Generator eingesetzt.
Vervollständigt die Funktion unten, so dass ein Generator entsteht. Die Parameter der Funktion sind:
 * d: tokenisierte und gesäuberte Tweets und Labels
 * w2i: das word2index dictionary
 * batch_size: Anzahl der vektorisierten Tweets, die pro Aufruf zurückgegeben werden sollen.
 
Die benutzen Tweets nacheinander aus `d` gewählt werden und kein Tweet mehrfach zurückgegeben werden.

In [23]:
shuffled = df.sample(frac=1, random_state=1).reset_index()
shuffled['tokenized']

0                                                                   [miss, nikki, nu, nu, already, shes, always, needed, thank, u, xxx]
1                             [So, I, dream, last, night., I, remember, sign, clearly, told, get, job., I, can't, rememer, sign, said.]
2                                                         [@girlyghost, ohh, poor, sickly, (((hugs)), hope, feel, little, better, soon]
3                                                                                                                             [raining]
4                                                                                                   [@MissKeriBaby, wish, I, LA, right]
                                                                       ...                                                             
1599995                                                                                   [@beckybootsx, hope, drinking, alcohol!, lol]
1599996    [That's, it!!, I, can't, take, more!!

In [24]:
import random
import numpy as np

def data_generator(x, y, w2i, batch_size):
  num_batches = np.ceil(len(x) / batch_size)
  current_batch = 0

  while True:
    # Initialize with zeros
    batch_x = np.zeros((batch_size, len(w2i.keys())))
    batch_y = np.zeros((batch_size, 1))

    start = current_batch * batch_size
    end = start + batch_size
    for idx, sentence in enumerate(x[start:end]):
      for word in sentence:
        word_index = w2i[str(word)]
        batch_x[idx][word_index] += 1
      batch_y[idx][0] = y[idx]
  
    current_batch += 1
    yield batch_x, batch_y

    # Reset counter
    if current_batch >= num_batches:
      current_batch = 0

In [25]:
gen = data_generator(shuffled['tokenized'], shuffled['polarity'], word2idx, BATCH_SIZE)

Ihr könnt euren Generator wie folgt ausprobieren:

In [26]:
next(gen)

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0

### 3.3 Neuronales Netz

Wir sind nun endlich soweit, unser neuronales Netz aufzubauen. Da unser Netz genau ein hidden Layer hat und auch sonst nicht sonderlich komplex ist, benutzen wir die _Sequential_-API von Keras.

In [27]:
from tensorflow.keras.models import Sequential
from keras.layers import Dense
m = Sequential()

Fügt einen _Dense_-Layer dem Netz hinzu, als _hidden units_ könnt ihr 16 nehmen. Da dies auch der Eingabe-Layer ist, müsst ihr den Parameter `input_shape` definieren. (Siehe auch: https://keras.io/layers/core/)

In [28]:
m.add(Dense(16, input_shape=(BATCH_SIZE, VECTOR_LEN), activation='relu'))

Als letzten Layer in unserem neuronalen Netz, fügt einen weiteren _Dense_-Layer hinzu. Dieser Layer dient auch als "Ausgabelayer" Überlegt euch die Anzahl der _hidden units_ (Hinweis: Wie lässt sich unser Machine-Learning-Problem kategorisieren?) Welche _Activation_-Funktion wählt ihr?

In [29]:
m.add(Dense(2, activation='softmax'))

7. Kompiliert das neuronale Netz. Als `optimizer` könnt ihr 'adam' benutzen. Wählt eine passende `loss`-Funktion aus. Begründet eure Entscheidung. (https://keras.io/models/model/#compile)

In [30]:
m.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### 3.3. Training

Bevor ihr nun das neuronale Netz trainiert, teilt noch euren Datensatz in zwei Teile auf. Einen Teil zum Trainieren und einen zum Evaluieren. Das Verhältnis der beiden Datensätze sollte 70%:30% sein. Bevor ihr die Daten aufteilen, durchmischt sie mit der `shuffle`-Methode aus dem `random`-Modul. Außerdem solltet ihr die Datenmenge zunächst auf ca. 100000 begrenzen, damit das Training des neuronalen Netzes nicht ewig dauert.

In [31]:
from sklearn.model_selection import train_test_split
training_data = shuffled[:100000]
train_x, val_x, train_y, val_y = train_test_split(training_data['tokenized'].to_list(), training_data['polarity'].to_list(), test_size=0.3, stratify=training_data['polarity'])
steps_per_epoch = np.ceil(len(train_x)/BATCH_SIZE)

In [32]:
print(train_x[:3])

[['@ButtonsBbyyy:', 'Sunshine', 'I', 'miss', 'baby!', 'Like', 'craazzyy!'], ['Thinks', 'gonna', 'rain'], ['Laughin', 'ethan..', 'U', 'ways', 'brighten', 'day']]


Wir sind nun soweit das neuronale Netz zu trainieren. Da wir den oben entwickelten Generator einsetzen wollen, verwenden wir dazu die `fit_generator`-Methode. Als `batch_size` könnt ihr 100 nehmen, für den `epochs`-Parameter 10. Was wählt ihr als `steps_per_epoch`-Parameter? (https://keras.io/models/model/#fit_generator)

In [43]:
#m.fit(data_generator(train_x, train_y, word2idx, BATCH_SIZE), epochs=10, steps_per_epoch=steps_per_epoch)
print(m.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100, 16)           21606800  
                                                                 
 dense_1 (Dense)             (None, 100, 2)            34        
                                                                 
Total params: 21,606,834
Trainable params: 21,606,834
Non-trainable params: 0
_________________________________________________________________
None


Während das Netz trainiert wird, könnt ihr euch Gedanken zur Evaluierung machen:
   * Definiert die üblichen Fehlerklassen (wahr positiv, falsch positiv, wahr negativ, falsch negativ)
   * Eine häufig benutzte Evaluationsmetrik ist die _Accuracy_. Beschreibt diese Metrik und schreibt die Formel zur Berechnung auf.
   * Warum könnte die _Accuracy_ eine schlechte Metrik sein?
   * Zur Evaluation von binären Klassifikationsproblemen wird in der Literatur gerne _Precision_ und _Recall_ verwendet. Wie sind die beiden Evaluationsmaße definiert? Beschreibt diese Metriken mit eigenen Worten. Schreibt auch die Formeln zur Berechnung auf.
   * Warum könnten _Precision_ und _Recall_ bessere Metriken sein als _Accuracy_?

11. Inzwischen sollte das Netz fertig trainiert sein. Speichert es ab!

In [34]:
#m.save('my_net.h5')

### 3.4. Evaluation

Evaluiert euer Netz mit dem Datensatz, den ihr oben beseite gelegt haben. Benutzt dafür die `predict`-Methode des Models. Berechnet dafür _Precision_, _Recall_, _Accuracy_ und _Confusion Matrix_. 

Interpretiert kurz eure Ergebnisse. 

In [36]:
#steps_per_epoch = int(np.ceil(len(val_x) / BATCH_SIZE))
#result = m.evaluate(data_generator(val_x,val_y,word2idx,BATCH_SIZE), steps=steps_per_epoch)

## 3b. (optional) Erweiterte Klassifikation
Im nächsten Schritt wollen wir das Netz auf dem vollen Datensatz trainieren und die Komplexität etwas erhöhen.

1. Trainiert euer Netz auf dem großen Datensatz.

In [ ]:
# TODO

2. Verändert die Parameter Ihres Netzes (z.B Anzahl _hidden units_, Anzahl _hidden layers_) und trainiert das Netz erneut (auf dem kleinen Datensatz). Was stellt ihr fest?

In [ ]:
# TODO

## 4. Sentiment Analyse mit Huggingface Transformers

In diesem Abschnitt werden wir nun Huggingface (https://huggingface.co/) zu Sentence Classification nutzen.

Hierfür benötigen wir die Libraries `datasets` (für preprocessing) und `transformers` (für die Transformer Modelle). Ebenfalls werden wir das Tokenisierung nicht mehr selbst durchführen, sondern viel mehr die `tokenizers` von huggingface hierfür benutzen.

Für eine gute Einführung in Klassifikation mit Transformers siehe: https://huggingface.co/docs/transformers/tasks/sequence_classification 

In [2]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 32.4 MB/s 
     |████████████████████████████████| 5.5 MB 56.9 MB/s 
     |████████████████████████████████| 212 kB 69.5 MB/s 
     |████████████████████████████████| 115 kB 72.9 MB/s 
     |████████████████████████████████| 163 kB 71.5 MB/s 
     |████████████████████████████████| 127 kB 67.9 MB/s 
     |████████████████████████████████| 7.6 MB 72.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [39]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.6 MB/s 


In [40]:
from datasets import ClassLabel, Value
from datasets import Dataset
import datasets
import pandas as pd
import numpy as np
import os
from transformers import Trainer, TrainingArguments
import evaluate

### 4.1 Data Preparation

Bevor wir ein huggingface transformer Modell nutzen können, müssen wir:

* den Datensatz (`df`) als `Dataset` laden
* ladet zunächst nur einen Teil des Datensatzes und wenn das Training funktioniert, vergrößert den Datensatz.
* die Spalten umbennenen 
* Train / Test Split (90 / 10)

1. Lade den Datensatz

In [5]:
! gdown 0B04GJPshIjmPRnZManQwWEdTZjg
DATA_DIR = '/content/sentiment140'
!unzip /content/trainingandtestdata.zip -d {DATA_DIR}
%rm /content/trainingandtestdata.zip
%ls -la /{DATA_DIR}

Downloading...
From: https://drive.google.com/uc?id=0B04GJPshIjmPRnZManQwWEdTZjg
To: /content/trainingandtestdata.zip
100% 81.4M/81.4M [00:00<00:00, 173MB/s]
Archive:  /content/trainingandtestdata.zip
  inflating: /content/sentiment140/testdata.manual.2009.06.14.csv  
  inflating: /content/sentiment140/training.1600000.processed.noemoticon.csv  
total 233292
drwxr-xr-x 2 root root      4096 Nov 16 13:20 ./
drwxr-xr-x 1 root root      4096 Nov 16 13:20 ../
-rw-r--r-- 1 root root     74326 Mar  4  2010 testdata.manual.2009.06.14.csv
-rw-r--r-- 1 root root 238803811 Mar  4  2010 training.1600000.processed.noemoticon.csv


In [6]:
column_names = ["polarity","id","date","query","user","text"]
dataset = pd.read_csv('/content/sentiment140/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', names=column_names)
dataset.drop(['date', 'query', 'user', 'id'], axis=1, inplace=True)
dataset["polarity"] = dataset["polarity"].apply(lambda x: 0 if x==0 else 1)
dataset = dataset.sample(frac=1, random_state=1).reset_index()
dataset.drop(['index'], axis=1, inplace=True)
dataset = dataset[:100000]

2. Entferne alle unnötigen Spalten und benennen `polarity` in `labels` um. Achte darauf, dass die Spalte `labels` vom Feature-Typ `ClassLabel` ist.

In [7]:
dataset = dataset.rename(columns={'polarity': 'label'})
dataset.head()

,label,text
0,0,i miss nikki nu nu already shes always there ...
1,0,So I had a dream last night. I remember a sig...
2,0,@girlyghost ohh poor sickly you (((hugs)) ho...
3,0,it is raining again
4,0,@MissKeriBaby wish I was in LA right now


In [8]:
dataset = Dataset.from_pandas(dataset)
new_features = dataset.features.copy()
new_features["label"] = ClassLabel(num_classes=1)
new_features["text"] = Value("string")
dataset = dataset.cast(new_features)

Casting the dataset:   0%|          | 0/100 [00:00<?, ?ba/s]

3. Teilt den Datensatz in train und test split auf.

In [9]:
split_dataset = dataset.train_test_split(test_size=0.1, shuffle=True)

Schaut euch die Klassenverteilung im Datensatz an? Habt ihr noch eine 50/50 Verteilung zwischen positiv und negativ?

In [10]:
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 90000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 10000
    })
})


### 4.1. Training
Nun haben wir den Datensatz geladen und vorbereitet. 

1. Ladet ein `AutoModelForSequenceClassification` eurer Wahl (https://huggingface.co/models) und den entsprechenenden `AutoTokenizer`.

  - Welches Modell nutzt ihr? Auf welchen Daten wurde das Netz vortrainiert?
  - Auf welchem Framework basiert das Modell?

In [11]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [12]:
from transformers import DistilBertForSequenceClassification
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

2. Bevor das eigentliche Training beginnt, muss der Datensatz noch tokenized werden. Nutzt hierfür den oben instantiierten `tokenizer` und beantwortet folgende Fragen:
  - Wie groß ist das des Tokenizers? 
  - In welcher Granularität werden Texte aufgesplittet?

In [13]:
import torch
from torch.utils.data import Dataset

class TwitterDataset(Dataset):
  def __init__(self, encodings, label):
        self.encodings = encodings
        self.label = label

  def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      item['label'] = torch.tensor(self.label[idx])
      return item

  def __len__(self):
      return len(self.label)


In [14]:
train = split_dataset['train']
test = split_dataset['test']
train_tok = tokenizer(train['text'], return_tensors='pt', padding=True, max_length=512, truncation=True)
test_tok = tokenizer(test['text'], return_tensors='pt', padding=True, max_length=512, truncation=True)

In [15]:
train_dataset = TwitterDataset(train_tok, train['label'])
test_dataset = TwitterDataset(test_tok, test['label'])

3. Baut euch eine Methode `compute_metrics`, die die notwendigen Metriken während des Trainings sammelt. Nutzt hierfür wieder Precision, Recall und F1 score.

In [16]:
print(train_dataset.__getitem__(0))

{'input_ids': tensor([  101,  1030,  1047, 21638, 10536,  2705,  2213,  2053,  2021,  1045,
         1005,  1049,  2514,  2378,  2204,  1012,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


In [55]:
def compute_metrics(eval_pred):
  """ Computes the metrics given a tuple of (logits, labels) """
  
  metrics = evaluate.combine([
    evaluate.load("accuracy", average="weighted"),
    evaluate.load("precision", average="weighted"),
    evaluate.load("f1", average="weighted"),
    evaluate.load("recall", average="weighted")
  ])

  metrics.compute(predictions=eval_pred[0], references=eval_pred[1], average="weighted")

4. Ladet einen `Trainer` und spezifiziert die `TrainingArguments`. Es bietet sich an, nach jeder Epoche zu evaluieren und das beste Modell am Ende zu laden.

In [18]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',           # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,    # tensorflow_datasets training dataset
    eval_dataset=test_dataset       # tensorflow_datasets evaluation dataset
)

5. Trainiert das Modell und behaltet die Metriken im Auge.

In [19]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 90000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 16875
  Number of trainable parameters = 66955010
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Step,Training Loss
500,0.523400
1000,0.449100
1500,0.435200
2000,0.423600
2500,0.417400
3000,0.414500
3500,0.397600
4000,0.388800
4500,0.404200
5000,0.392700


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
Saving mode

TrainOutput(global_step=16875, training_loss=0.2942911752206308, metrics={'train_runtime': 4195.633, 'train_samples_per_second': 64.353, 'train_steps_per_second': 4.022, 'total_flos': 1.2224774583e+16, 'train_loss': 0.2942911752206308, 'epoch': 3.0})

### 4.3. (Optional) Training mit weiterem Modell

Falls ihr möchtet, könnt ihr optional das geladene vortrainierte Modell austauschen und ein anderes (größeres, auf anderen Daten trainiert, ...) Modell nutzen.


### 4.4. Prediction

Nun könnt ihr das Modell nutzen, um ein paar Beispiel aus dem Test-Datensatz zu predicten.

In [24]:
id2label = { 0: "neg", 1: "pos" }
prediction_ds = test_dataset

Gebt die falschen Predictionn aus. Was fällt euch auf?

In [35]:
raw_pred, _, _ = trainer.predict(prediction_ds)
print(f"Total predictions: {len(raw_pred)}")

***** Running Prediction *****
  Num examples = 10000
  Batch size = 64
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


Total predictions: 10000


In [36]:
wrong_counter = 0
for text, prediction in zip(prediction_ds, np.argmax(raw_pred, axis=1)):
  expected = text['label'].item()
  if expected != prediction:
    wrong_counter +=1
    #print(f"Expected: {id2label[expected]} --- Predicted: {id2label[prediction]}")

print(f"number of wrong predictions out of 10k: {wrong_counter}")
print(f"resulting Accuracy: {1 - wrong_counter / 10000}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


number of wrong predictions out of 10k: 1654
resulting Accuracy: 0.8346


In [56]:
compute_metrics(raw_pred)

ValueError: ignored

https://discuss.huggingface.co/t/combining-metrics-for-multiclass-predictions-evaluations/21792